In [1]:
import time, scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
import scipy.signal
import time
import freeDeconvolution
#import subordination, sampling, elkaroui
import multiprocessing as mp
import json

T = np.arange(0,10, 0.2)
nu = np.array( [complex(0.1*a,b) for a in range(0  , 10) for b in [0.01,0.1]] )

# Returns Scenario in the form of p eigenvalues
def gen_matrix(Scenario, p):
        np.random.seed(seed=None)
        if Scenario=="Case1":
            population_spectrum = np.ones( p )
            weights = np.array( [1], dtype=np.float64 )
            support = np.array( [1], dtype=np.float64 )

        elif Scenario=="Case2" or Scenario=="Case2.2" or Scenario=="Case2.3" :
            if Scenario=="Case2":
                weights = np.array( [1, 1], dtype=np.float64 )
                support = np.array( [1, 2], dtype=np.float64 )
            elif Scenario=="Case2.2" :
                weights = np.array( [1, 1], dtype=np.float64 )
                support = np.array( [1, 1.3], dtype=np.float64 )
            else :
                weights = np.array( [1, 1, 1, 1, 1], dtype=np.float64 )
                support = np.array( [1, 2, 3, 5, 6], dtype=np.float64 )
            weights = weights/np.sum( weights )
 
            population_cdf = np.cumsum( weights )

            population_spectrum = np.zeros( (p,) )
            block_begin = 0
            for i in range( len(weights) ):
                block_end = int( population_cdf[i]*p )
                population_spectrum[block_begin:block_end] = support[i]
                block_begin = block_end

        elif Scenario=="Case3":
            c = 1
            indices = np.arange( 0, p, 1)
            toeplitz_row    = 0.3**indices
            toeplitz = scipy.linalg.toeplitz( toeplitz_row)
            
            population_spectrum, U = np.linalg.eig(toeplitz)
            population_spectrum = np.sort( population_spectrum )

            weights = np.ones(p)*1.0/p
            support = population_spectrum

        else:
            print( "Please specify a scenario..." )
            raise Error()
        
        return (population_spectrum, weights, support)

# I. Generate experiments JSON

In [2]:
N_range   = [ 256, 512, 1024, 2048]
nb_itr    = 5
Scenarios = [ "Case1", "Case2", "Case2.2", "Case2.3", "Case3"]
#Scenarios = [ "Case2.3"]

DOEs = []

# Loop over scenarios
for Scenario in Scenarios:
    print( f'''  |- Scenario {Scenario}''')
    # Loop over iterations
    for iteration in range(nb_itr):
        # Loop over N
        for N in N_range:
            T = np.arange(0,10, 0.2)
            nu = np.array( [complex(0.1*a,b) for a in range(0  , 10) for b in [0.01,0.1]] )
            c = 1
            p = int(c*N)

            generated_data = gen_matrix(Scenario, p)
            population_spectrum, weights, support = generated_data
            diag           = freeDeconvolution.sampling.sample_wishart( p, N, population_spectrum )

            population_cdf = np.zeros_like( T )
            for i in range( len(T) ):
                t = T[i]
                population_cdf[i] = np.count_nonzero( population_spectrum <= t )
            population_cdf = population_cdf/p
            #
            DOEs.append( {
                "Scenario": Scenario,
                "N"       : N,
                "observed_spec"    : list(diag),
                "population_spec"  : list(population_spectrum),
                "population_cdf"   : list(population_cdf),
                "truth_weights"    : list(weights),
                "truth_support"    : list(support),
                "results_by_method": {}
            } )
        #
    #
#

  |- Scenario Case1
  |- Scenario Case2
  |- Scenario Case2.2
  |- Scenario Case2.3
  |- Scenario Case3


In [3]:
with open("./DOEs.json", "w") as f:
    json.dump( DOEs, f, indent=4)

In [4]:
# WARNING: Crushes old results
with open("./DOEs_with_results.json", "w") as f:
    json.dump( DOEs, f, indent=4)

# II. Loading and processing DOEs

## II.1. El Karoui's method

In [5]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

Loaded 100 experiments...


In [6]:
methods = [ "convex_optim", "subordination", "our method"]
method  = methods[0]

In [7]:
def compute_DOE_with_convex( DOE, norm="l2"):
    c = 1
    T = np.arange(0,10, 0.2)
    nu = np.array( [complex(0.1*a,b) for a in range(0  , 10) for b in [0.01,0.1]] )

    # RMT data
    diag = np.array( DOE["observed_spec"] )
    population_spectrum = np.array( DOE["population_spec"] )
    population_cdf      = np.array( DOE["population_cdf"] )

    ## El Karoui
    tic = time.time()
    Z = freeDeconvolution.elkaroui.build_dictionary( nu, c, diag)
    nu_check = -(1-c)/Z + c*freeDeconvolution.elkaroui.stieltjes(Z, diag)
    nu_errors = np.abs(nu - nu_check)
    bad_indices = np.where(nu_errors > 1e-5)

    # Clean-up if necessary
    if len(bad_indices):
        Z  = np.delete( Z , bad_indices )
        nu = np.delete( nu, bad_indices )
    dictionary = (Z, nu)

    # Perform optimization
    weights_convex, objective_value = freeDeconvolution.elkaroui.perform_cvx_optimization( dictionary, T, c, norm, verbose=False)
    weights_convex = abs(weights_convex)
    toc    = time.time()
    timing = toc-tic
    print( f'''Timing for convex optimization by el Karoui {str(timing)}''' )

    ## Done
    error   = wasserstein_distance( T, population_spectrum,  weights_convex, np.ones(len(population_spectrum))/len(population_spectrum))
    new_DOE = DOE.copy()
    new_DOE["results_by_method"]["convex_optim"] = {
        "error": error,
        "timing": timing,
        "weights": list(weights_convex),
        "support": list(T)
    }

    return new_DOE

In [8]:
# Loop over experiments
print( "Loop over DOEs using multiprocessing... ")
num_processes = 12 # Use all the available CPU cores of computer
with mp.Pool(processes=num_processes) as pool:
    results = pool.map( compute_DOE_with_convex , DOEs)

Loop over DOEs using multiprocessing... 
Timing for convex optimization by el Karoui 3.8500752449035645Timing for convex optimization by el Karoui 3.8619394302368164

Timing for convex optimization by el Karoui 3.8928749561309814
Timing for convex optimization by el Karoui 3.897927761077881
Timing for convex optimization by el Karoui 3.9428532123565674
Timing for convex optimization by el Karoui 3.971294403076172
Timing for convex optimization by el Karoui 4.006690502166748


/home/rchhaibi/CodeProjects/free_deconvolution/.venv_freeDeconvolution/lib/python3.10/site-packages/cvxpy-1.2.2-py3.10-linux-x86_64.egg/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Timing for convex optimization by el Karoui 4.02618145942688
Timing for convex optimization by el Karoui 4.069420337677002
Timing for convex optimization by el Karoui 4.105071783065796
Timing for convex optimization by el Karoui 4.144397735595703
Timing for convex optimization by el Karoui 4.261013746261597
Timing for convex optimization by el Karoui 3.7336695194244385
Timing for convex optimization by el Karoui 3.83510160446167

/home/rchhaibi/CodeProjects/free_deconvolution/.venv_freeDeconvolution/lib/python3.10/site-packages/cvxpy-1.2.2-py3.10-linux-x86_64.egg/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(



Timing for convex optimization by el Karoui 3.91426420211792
Timing for convex optimization by el Karoui 3.938027858734131Timing for convex optimization by el Karoui 3.861032247543335

Timing for convex optimization by el Karoui 3.8958773612976074Timing for convex optimization by el Karoui 3.8593685626983643

Timing for convex optimization by el Karoui 4.048290729522705Timing for convex optimization by el Karoui 3.9687461853027344

Timing for convex optimization by el Karoui 4.1863861083984375
Timing for convex optimization by el Karoui 4.1557722091674805
Timing for convex optimization by el Karoui 4.436791658401489
Timing for convex optimization by el Karoui 3.97460675239563
Timing for convex optimization by el Karoui 3.8299877643585205
Timing for convex optimization by el Karoui 3.9565773010253906
Timing for convex optimization by el Karoui 4.1789350509643555Timing for convex optimization by el Karoui 3.858586311340332

Timing for convex optimization by el Karoui 4.118809700012207
T

...


Timing for convex optimization by el Karoui 4.494732856750488
Timing for convex optimization by el Karoui 3.987393379211426
Timing for convex optimization by el Karoui 4.575291395187378
Timing for convex optimization by el Karoui 4.235907554626465


/home/rchhaibi/CodeProjects/free_deconvolution/.venv_freeDeconvolution/lib/python3.10/site-packages/cvxpy-1.2.2-py3.10-linux-x86_64.egg/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Timing for convex optimization by el Karoui 3.3508193492889404
Timing for convex optimization by el Karoui 3.8293752670288086
Timing for convex optimization by el Karoui 3.6920177936553955
Timing for convex optimization by el Karoui 3.478788137435913
Timing for convex optimization by el Karoui 3.465437412261963
Timing for convex optimization by el Karoui 3.5158543586730957
Timing for convex optimization by el Karoui 3.556635856628418
Timing for convex optimization by el Karoui 3.6171133518218994
Timing for convex optimization by el Karoui 3.9611246585845947
Timing for convex optimization by el Karoui 3.7538933753967285
Timing for convex optimization by el Karoui 3.440082550048828
Timing for convex optimization by el Karoui 3.0978071689605713
Timing for convex optimization by el Karoui 3.587603807449341
Timing for convex optimization by el Karoui 2.9437811374664307


/home/rchhaibi/CodeProjects/free_deconvolution/.venv_freeDeconvolution/lib/python3.10/site-packages/cvxpy-1.2.2-py3.10-linux-x86_64.egg/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Timing for convex optimization by el Karoui 3.731963634490967
Timing for convex optimization by el Karoui 3.6886942386627197
Timing for convex optimization by el Karoui 3.101240396499634
Timing for convex optimization by el Karoui 3.5591795444488525
Timing for convex optimization by el Karoui 3.510215997695923
Timing for convex optimization by el Karoui 3.4897828102111816
Timing for convex optimization by el Karoui 3.6069154739379883
Timing for convex optimization by el Karoui 2.757227897644043
Timing for convex optimization by el Karoui 2.8497159481048584
Timing for convex optimization by el Karoui 3.066683292388916
Timing for convex optimization by el Karoui 3.439594030380249
Timing for convex optimization by el Karoui 3.6065924167633057
Timing for convex optimization by el Karoui 3.153543710708618
Timing for convex optimization by el Karoui 2.9149951934814453


In [9]:
with open("./DOEs_with_results.json", "w") as f:
    json.dump( results, f, indent=4)

## II.2. Tarrago's method

In [10]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

Loaded 100 experiments...


In [11]:
## II.2. Processing with the subordination method
methods = [ "convex_optim", "subordination"]
method  = methods[1]

In [12]:
def compute_DOE_with_subordination( DOE):
    # RMT data
    diag = np.array( DOE["observed_spec"] )
    population_spectrum = np.array( DOE["population_spec"] )
    population_cdf      = np.array( DOE["population_cdf"] )
    
    ## Tarrago
    tic  = time.time()
    y, R = freeDeconvolution.subordination.freedeconvolutionresult(diag)
    weights_subordination = abs(R)/np.sum(R)
    toc    = time.time()
    timing = toc-tic
    print( f'''Timing for convex optimization by Tarrago {str(timing)}''' )

    ## Done
    error   = wasserstein_distance( y, population_spectrum,  weights_subordination, np.ones(len(population_spectrum))/len(population_spectrum))
    new_DOE = DOE.copy()
    new_DOE["results_by_method"]["subordination"] = {
        "error"  : error,
        "timing" : timing,
        "weights": list(weights_subordination),
        "support": list(y)
    }

    return new_DOE

In [13]:
# Loop over experiments
print( "Loop over DOEs using multiprocessing... ")
num_processes = 12 # Use all the available CPU cores of computer
with mp.Pool(processes=num_processes) as pool:
    results = pool.map( compute_DOE_with_subordination , DOEs)

Loop over DOEs using multiprocessing... 
 Tarrago : step1 = 0.3866126537322998 step2 0.6775779724121094
Timing for convex optimization by Tarrago 1.1103484630584717
 Tarrago : step1 = 0.38277769088745117 step2 0.9994471073150635
Timing for convex optimization by Tarrago 1.4249062538146973
 Tarrago : step1 = 0.5532641410827637 step2 1.2596094608306885
Timing for convex optimization by Tarrago 1.8971397876739502
 Tarrago : step1 = 0.9599087238311768 step2 1.2729275226593018
Timing for convex optimization by Tarrago 2.2708792686462402
 Tarrago : step1 = 1.3611557483673096 step2 1.1285922527313232
Timing for convex optimization by Tarrago 2.533825159072876
 Tarrago : step1 = 1.4634580612182617 step2 1.3526978492736816
 Tarrago : step1 = 1.505237102508545 step2 1.3138461112976074Timing for convex optimization by Tarrago 2.8872859477996826
Timing for convex optimization by Tarrago 2.9175240993499756

 Tarrago : step1 = 2.556507110595703 step2 0.8000776767730713
Timing for convex optimization

In [14]:
with open("./DOEs_with_results.json", "w") as f:
    json.dump( results, f, indent=4)

## II.3. Our method

In [15]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

Loaded 100 experiments...


In [16]:
## II.3. Processing with our method
methods = [ "convex_optim", "subordination", "our method"]
method  = methods[2]

In [17]:
def compute_DOE_with_our_method( DOE, debug=False, debug_aggressive=False, plot=True):
    c = 1
    N = DOE['N']
    p = int(c*N)
        
    # RMT data
    diag = np.array( DOE["observed_spec"] )
    population_spectrum = np.array( DOE["population_spec"] )
    population_cdf      = np.array( DOE["population_cdf"] )
    
    tic  = time.time()

    ## Init
    mu_observed = freeDeconvolution.core.DiscreteMeasure( diag, None)
    mu_signal   = freeDeconvolution.core.DiscreteMeasure( population_spectrum, None)

    mu_observed.compute_second_kind()

    zeroes_first_kind   = mu_observed.zeroes_first_kind
    zeroes_second_kind  = mu_observed.zeroes_second_kind

    ## Find bounding box
    from freeDeconvolution import boxes
    degree = len(diag)

    if debug:
        print("")
        print("-----------------------------------")
        print("id       : ", DOE['id'])
        print("Scenario : ", DOE['Scenario'])
        print("")

    mesh_size = 10000
    # radius = np.max(diag)/2 + 1
    # center = np.max(diag)/2
    # interval = np.linspace(0, 2*np.pi, mesh_size)
    # contour = center + radius*( np.cos(interval) + np.sin(interval)*1.0j)
    # plt.scatter( np.real(diag), np.imag(diag), c='r')
    # plt.plot( np.real(contour), np.imag(contour) )
    # plt.show()

    def index_integrand(z):
        values = mu_observed.Markov_Krein_prime(z)/mu_observed.Markov_Krein(z)
        return values

    # values = index_integrand(contour)
    # dz = 1.0j*(contour-center)*2*np.pi/(mesh_size) 
    # index  = np.sum(dz*values)/(2*np.pi*1.0j)
    # print( "Index: ", index)
    # print( "Root count: ", index+2*degree)

    # print( "Box segments enumeration: ")
    # print( boxes.box_segments_enum )

    def compute_index( box, mesh_size, plot=True, color='b'):
        interval =  np.linspace( 0,1, mesh_size)
        integral = 0
        for segment in boxes.box_segments_enum:
                vector = box[ segment[1] ] - box[ segment[0] ]
                origin = box[ segment[0] ]
                s = origin + interval*vector
                #
                values = index_integrand(s)
                dz = ( s[-1]-s[0] )/mesh_size
                integral = integral + np.sum( values*dz )
        return integral/(2*np.pi*1.0j)
    # TODO: Make it more versatile. Here tuning by hand of radius.

    radius = 4
    mesh_size = int(1e4)
    bounding_box = {
        'top_left'    : np.min(diag) - 0.3 + radius*1.0j,
        'bottom_right': np.max(diag) + 0.3 - radius*1.0j,
    }
    bounding_box   = boxes.extend_box(bounding_box)
    index = compute_index( bounding_box, mesh_size)
    index = np.real(index+2*degree)
    root_count = int( np.round( index ) )
    error = index-root_count
    if debug:
        print( "Checking the number of critical points in initial bounding box")
        print( "Index of contour : ", index)
        print( "Root count: ", "2x", 0.5*root_count)
        print( "p         : ", p)
        print( "")

    # If false, the bounding box missed roots
    assert( p-1 == int(0.5*root_count))
    
    ## Find small contour
    find_small_contour = True
    if not find_small_contour:
        if debug:
            print("Using default bounding box...")
        contour_height = radius
    else:
        if debug:
            print("Finding near-optimal bounding box...")
        #
        box = bounding_box.copy()
        box['bottom_left'] = box['top_left'] # For initialization, bottom_left needs to be the previous top_left
        radius = box['height']/2
        stop_at_first_nonempty = True        # Stop at first found box with roots

        # Loop for multiple passes and more
        boxes_with_roots = []
        boxes_with_roots.append( box )
        root_counter = 0
        total_roots  = p-1 # Total number of roots in upper half plane
        i = 0
        while( root_counter < total_roots ):
            i = i+1
            radius = radius/2
            box = {
                    'top_left'    : box['bottom_left'],
                    'bottom_right': np.real( box['bottom_right'] ) + radius*1.0j,
            }
            box   = boxes.extend_box(box)
            index = compute_index( box, mesh_size, plot=False)
            index = np.real(index)
            root_count = int( np.round( index ) )
            root_counter = root_counter + root_count
            error = index-root_count
            if debug_aggressive:
                print(f"Pass {i}:")
                print( "Index: ", index)
                print( "Root count / Total: ", root_count, '/', root_counter)
                print( "Found:", root_counter, " / ", total_roots )
                print( "")
            #
            if root_count>0:
                    box['root_count'] = root_count
                    boxes_with_roots.append( box )
                    if stop_at_first_nonempty:
                            break
        # end  while
        contour_height = 2*radius
    # end if find_small_contour

    ## Setup arrays
    contour_type = "rectangle"
    mesh_size    = int(1e4)
    left_point   = np.min(diag)
    right_point  = np.max(diag)
    mid_point    = 0.5*left_point + 0.5*right_point
    
    # New bounding box
    print( "Contour height: ", contour_height )
    box = {
            'top_left'    : np.real( bounding_box['top_left'],    ) - 1.0 + contour_height*1.0j,
            'bottom_right': np.real( bounding_box['bottom_right'] ) + 1.0 - contour_height*1.0j,
    }
    box   = boxes.extend_box(box)
    print( "Bounding box height (new): ", box )
    bounding_box = box


    if contour_type=="rectangle":
        path = boxes.box_to_path( bounding_box, mesh_size )
        z_array = np.array( path )
    elif contour_type=='circle':
        radius = np.maximum( 1.1*(right_point-left_point)/2 , bounding_box["height"] )
        #
        interval = np.linspace(0, 2*np.pi, mesh_size)
        z_array = mid_point + radius*( np.cos(interval) + np.sin(interval)*1.0j)
    
    m_array = mu_observed.M_empirical( z_array )
    s_array = (1+m_array)/(m_array*z_array)

    # Various arrays
    s_signal_array = s_array
    s_noise_array  = 1/(c*m_array + 1)
    s_deconv_array = s_signal_array/s_noise_array
    #
    # NEXT LINE IS F*** UP
    # m_deconv_array = 1/( s_deconv_array*z_array - 1)
    #
    # TENTATIVE FIX
    m_deconv_inv_array = (1+1/m_array)/s_deconv_array
    m_deconv_array   = m_array
    z_array_original = z_array
    z_array          = m_deconv_inv_array
    #
    #m_deconv_theoretical_array = mu_signal.M_empirical( z_array )
    g_deconv_array = (m_deconv_array+1)/z_array
    #g_deconv_theoretical = (m_deconv_theoretical_array+1)/z_array


    ## Plot of newly found contour
    if plot:
        plt.figure()
        plt.rcParams["figure.figsize"] = (5, 5)
        plt.plot( np.real(z_array), np.imag(z_array), label="Transformed z contour" )
        plt.plot( np.real(z_array_original), np.imag(z_array_original), label="Original z contour" )
        plt.scatter( zeroes_first_kind, np.zeros_like(zeroes_first_kind), label="Roots of $mu_n$")
        plt.title( f'''Contour for DOE id={DOE['id']}''' )
        plt.legend()
        plt.savefig(f'''contour_id_{DOE['id']}.png''' )

    ## Compute moments
    dz_array  = z_array-np.roll(z_array, shift=1)
    # Rectangle Riemann integration
    def cauchy_integral_g_deconv( f ):
        value = g_deconv_array*f*dz_array
        #value = g_deconv_theoretical*f*dz_array
        return value.sum()/(2*np.pi*1.0j)
    # Trapezoid Riemann integration
    def cauchy_integral_trapezoid_g_deconv( f ):
        integrand = g_deconv_array*f
        value = 0.5*( integrand + np.roll(integrand, shift=1) )*dz_array
        return value.sum()/(2*np.pi*1.0j)
    moments_count = 5
    mom_array       = np.zeros( 2*moments_count + 2)
    mom_array_truth = np.zeros( 2*moments_count + 2)
    truth_weights   = np.array( DOE['truth_weights'] )
    truth_support   = np.array( DOE['truth_support'] )
    centering       = 0
    for mom_index in range( len(mom_array) ):
        value = cauchy_integral_trapezoid_g_deconv( (z_array-centering)**mom_index ) # Centering stabilizes
        mom_array[ mom_index ] = np.real(value)
        # Activate centering
        # if mom_index==1:
        #      centering    = mom_array[1]
        #      mom_array[1] = 0
        mom_array_truth[mom_index] = np.dot( truth_weights, (truth_support-centering)**mom_index)
    # end for

    if debug:
        np.set_printoptions(precision=5, suppress=True)
        #print( "Midpoint: ", mid_point )
        print( "Estimated moments: ")
        print( mom_array )
        print( "Ground truth:")
        print( mom_array_truth )
        print( "Relative error:")
        print( (mom_array-mom_array_truth)*100/mom_array_truth )
        #np.set_printoptions(suppress=False)
        print( "")

    ## Inverse moment problem
    print( "Performing inverse moment problem...")
    jacobi_a, jacobi_b = freeDeconvolution.oprl.jacobi_from_moments( mom_array )
    support, weights   = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
    support = support + centering
    non_outliers = np.where( (support>left_point)*(support<right_point) )[0]
    support = support[non_outliers]
    weights = weights[non_outliers]
    weights = weights/np.sum(weights)

    toc    = time.time()
    timing = toc-tic
    print( f'''Timing for our method {str(timing)}''' )

    ## Done
    error   = wasserstein_distance( support, population_spectrum, weights, np.ones(len(population_spectrum))/len(population_spectrum))
    new_DOE = DOE.copy()
    new_DOE["results_by_method"]["our method"] = {
        "error"  : error,
        "timing" : timing,
        "weights": list(weights),
        "support": list(support)
    }

    with open( f'''./dump/DOE_{DOE['id']}.json''', "w") as f:
        json.dump( new_DOE, f, indent=4)

    print( "Error  : ", error)
    print( "Weights: ", weights)
    print( "Support: ", support)
    print( "" )
    return new_DOE

In [18]:
import os
import string

# Tag DOEs for identification if bugs
for i in range( len(DOEs) ):
    DOE       = DOEs[i]
    DOE['id'] = 31000+i

# Lazy mode
lazy = False
if lazy:
    print( "Working in lazy mode...")
    files = os.listdir('./dump')
    str_identifiers = [ f.strip(string.ascii_letters)[1:-1] for f in files]
    identifiers     = sorted( [ int(str_id)-31000 for str_id in str_identifiers] )
    print( "Found files: ", identifiers )
    all_indices      = set( range(len(DOEs)) )
    selected_indices = all_indices.difference( identifiers )
    selected_indices = list(selected_indices)[:]
    print( f'''Number of selected indices: {len(selected_indices)}/{len(DOEs)}''')
    print( "Selected indices:", selected_indices)
    selected_DOEs = [ DOEs[i] for i in selected_indices ]
    print( "")
else:
    selected_DOEs = DOEs

# Loop over experiments
print( "Loop over DOEs using multiprocessing... ")
num_processes = 8 # Use all the available CPU cores of computer
with mp.Pool(processes=num_processes) as pool:
    results = pool.map( compute_DOE_with_our_method, selected_DOEs)
# for DOE in selected_DOEs:
#     compute_DOE_with_our_method( DOE )

Loop over DOEs using multiprocessing... 

-----------------------------------
id       :  31000
Scenario :  Case1

Checking the number of critical points in initial bounding box
Index of contour :  510.00010921477053
Root count:  2x 255.0
p         :  256

Finding near-optimal bounding box...
Contour height:  0.25
Bounding box height (new):  {'top_left': (-1.2999995658395365+0.25j), 'bottom_right': (5.182797976667346-0.25j), 'top_right': (5.182797976667346+0.25j), 'bottom_left': (-1.2999995658395367-0.25j), 'height': 0.5, 'width': 6.482797542506883}
Estimated moments: 
[ 1.       0.99131  0.98358  0.9872   1.00982  1.04365  1.04892  0.93509
  0.53837 -0.39562 -2.18083 -5.01244]
Ground truth:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Relative error:
[  -0.        -0.86944   -1.6421    -1.27988    0.98204    4.36517
    4.89176   -6.49128  -46.16257 -139.56213 -318.08331 -601.24402]

Performing inverse moment problem...
3-th principal minor is not positive definite
Timing for our method 3.95

KeyboardInterrupt: 

In [ ]:
with open("./DOEs_with_results.json", "w") as f:
    json.dump( results, f, indent=4)

## II.4. Fusion

In [ ]:
import os
import string

files = os.listdir('./dump')
DOEs_fused = []
for filename in files:
    f = open( f'''./dump/{filename}''', 'r')
    DOE = json.load( f )
    DOEs_fused.append( DOE )
# end for

with open("./DOEs_fused.json", "w") as f:
    json.dump( DOEs_fused, f, indent=4)


# III. Plotting

In [ ]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

In [ ]:
methods = [ "convex_optim", "subordination", "our method"]

aggregated_results = freeDeconvolution.plots.aggregate_benchmarks( DOEs, methods)
freeDeconvolution.plots.make_plots( aggregated_results, methods )

# IV. Analysis

Here we dump data to csv file for further analysis

In [ ]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

In [ ]:
extracts = []
for DOE in DOEs:
    result   = DOE['results_by_method']['our method']
    error    = result['error']
    timing   = result['timing']
    extract = [ DOE['Scenario'], DOE['N'], error, timing, result["weights"], result["support"] ]
    extracts.append( extract )
# end for

import csv

col_names = ["Scenario", "N", "error", "timing", "weights", "support"]

with open("./extracts.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow( col_names )
    writer.writerows(extracts)